# Introduction to LSTMs
Long Short-Term Memory networks, usually abbreviated in LSTM, are a kind of Recurrent Neural Network proposed by Hochreiter & Schmidhuber in their 1997 article ["Long Short-Term Memory"](http://www.bioinf.jku.at/publications/older/2604.pdf).

LSTMs were designed in order to remember the data given in input for long periods of time, while avoiding issues afflicting other types of RNNs like the *vanishing gradient*, the *exploding gradient* the *long-term dependancy* problems.

The *vanishing gradient* problem consists in the fact that RNNs, by repeatedly calculating the value of a gradient used to update the weights of the model, tend to make the value of such gradient converge to zero, thus preventing the network to learn effectively from the input data.

The *exploding gradient* problem is similar to the vanishing gradient one, but in this case the value of the gradient tends to be updated to progressively bigger values, thus making the value of the gradient so high that the model becomes unstable and unable to learn from the training data.

The *long-term dependency* problem consists in the inability of detecting associations between data points which are distant from each other in the input sequence.

LSTM networks address the aforementioned problems by exploiting in each of its units (also called *cells*), an *input gate*, an *output gate* and a *forget gate*. While the cell remembers values over arbitrary time intervals, the three gates regulate the flow of information inside the cell and which information forward to the other cells.

At the time **t** a LSTM cell takes in input the hidden layer **h** and the cell state **c** calculated by the LSTM at time **t-1** and the t-th portion of the input sequence **x**. The forget gate enstablishes which parts of the previous cell and hidden state to use in the new cell state, the input gate decides the extent to which a new value of the input sequence flows into the new cell state, and the ouput gate determines how to calculate the new hidden state of the LSTM cell (which will also be used to calculate the output **y** of the cell.

![image](https://drive.google.com/uc?id=104-iQvZTMBMjilbJHSANFoMrM8_hMFil)



# Time series
In this tutorial we will use LSTM networks in order to make predictions on time series data. Time series are a set of data points indexed in time order; such data points are obtained through a series of observations on the same sets of variables in successive points in time (points which are typically but not always equally spaced). Time series are thus a sequence of discrete-time data.
If a time series contains observations regarding only one variable then it is called a **univariate time series**; in case a time series contains observations regarding a set of two or more variable then it is called a **multivariate time series**.

In this practice lesson we will use a LSTM to predict the daily cases of COVID-19 in the world through an univariate time series. To do so we'll use the dataset "csse_covid_19_time_series", which contains the following features:
  - "Province/State" the province or state from which the row of data is from
  - "Country/Region", the country or region of the province or state the data refers to;
  - "Lat", the latitude of the country/region;
  - "Long", the longitude of the country/region;
  - a series of dates with the number of cases in that day in a province/state. 

Let's first open the dataset and take a look at it.

In [1]:
import pandas as pd

ds = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
print(ds)

    Province/State      Country/Region        Lat        Long  1/22/20  \
0              NaN         Afghanistan  33.939110   67.709953        0   
1              NaN             Albania  41.153300   20.168300        0   
2              NaN             Algeria  28.033900    1.659600        0   
3              NaN             Andorra  42.506300    1.521800        0   
4              NaN              Angola -11.202700   17.873900        0   
..             ...                 ...        ...         ...      ...   
268            NaN             Vietnam  14.058324  108.277199        0   
269            NaN  West Bank and Gaza  31.952200   35.233200        0   
270            NaN               Yemen  15.552727   48.516388        0   
271            NaN              Zambia -13.133897   27.849332        0   
272            NaN            Zimbabwe -19.015438   29.154857        0   

     1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  ...  1/14/21  1/15/21  \
0          0        0        0      

As it can seen by printing the dataset the data needs to be cleaned, hence the following steps are needed:
  1. drop the first 4 columns because unneeded;
  2. sum all rows to get a new dataset composed by the cumulative daily cases, this in order to pass from a multivariate time series to an univariate one;
  3. check for missing values in the daily cases dataset and eliminate eventual rows with them;
  4. split the daily cases dataset by using the first 200 days as training set and the remaining 68 as testing set;
  5. scale the data with the MinMax scaler.


In [2]:
### Step 1. ###
ds = ds.iloc[:, 4:] ### CODE HERE ###

### Step 2. ###
daily_cases = ds.sum(axis=0) ### CODE HERE ###
daily_cases.index = pd.to_datetime(daily_cases.index) ### CODE HERE ###
print(daily_cases.head)

### Step 3. ###
number_of_nans = daily_cases.isnull().sum().sum() ### CODE HERE ###
print(number_of_nans)

### Step 4. ###
train_data = daily_cases[:-68] ### CODE HERE ###
test_data = daily_cases[-68:] ### CODE HERE ###

### Step 5. ###
import numpy as np
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler = scaler.fit(np.expand_dims(train_data, axis=1))  # expand_dims is needed to insert a new axis during the scaling operation to assure that the MinMaxScaler performs correctly

train_data = scaler.transform(np.expand_dims(train_data, axis=1)) ### CODE HERE ###
test_data = scaler.transform(np.expand_dims(test_data, axis=1)) ### CODE HERE ###

<bound method NDFrame.head of 2020-01-22         557
2020-01-23         655
2020-01-24         941
2020-01-25        1433
2020-01-26        2118
                ...   
2021-01-19    96167933
2021-01-20    96862056
2021-01-21    97518881
2021-01-22    98177108
2021-01-23    98746982
Length: 368, dtype: int64>
0


Currently we have a big sequence of daily cases so we’ll convert it into smaller ones (of length seq_length=5) in order to ease the training process of our model.

In [3]:
import torch

def create_sequences(data, seq_length=5):
    
    xs = []  # data sequence
    ys = []  # target sequence

    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    
    return np.array(xs), np.array(ys)

X_train, y_train = create_sequences(train_data) ### CODE HERE ###  # creating the training data sequences
X_test, y_test = create_sequences(test_data) ### CODE HERE ###  # creating the test data sequences

# casting the sequences to float numbers
X_train = torch.from_numpy(X_train).float() ### CODE HERE ###
y_train = torch.from_numpy(y_train).float() ### CODE HERE ###

X_test = torch.from_numpy(X_test).float()### CODE HERE ###
y_test = torch.from_numpy(y_test).float()### CODE HERE ###

Now it's time to incapsulate the model in a torch class. We'll make a model in which a linear layer takes in input the data calculated by a LSTM network. The class will have as parameters the number of features in input, the number of hidden layers, the length of the sequence and the number of LSTM layers.

In [4]:
import torch.nn as nn

class TimeSeriesPredictor(nn.Module):

  def __init__(self, n_features=1, n_hidden=512, seq_len=5, n_layers=1, out_features=1):
    super(TimeSeriesPredictor, self).__init__()

    self.n_hidden = n_hidden ### CODE HERE ###
    self.seq_len = seq_len ### CODE HERE ###
    self.n_layers = n_layers### CODE HERE ###
    self.out_features = out_features ### CODE HERE ###

    self.lstm = nn.LSTM(input_size=n_features, hidden_size=n_hidden, num_layers=n_layers) ### CODE HERE ###
    
    self.linear = nn.Linear(in_features=n_hidden, out_features=self.out_features) ### CODE HERE ###

  def forward(self, sequences):
    # output of the LSTM and updated hidden state of the LSTM
    lstm_out, self.hidden = self.lstm(sequences.view(len(sequences),
                                                     self.seq_len, -1),
                                      self.hidden)
    
    # last time step handled by the LSTM
    last_time_step = lstm_out.view(self.seq_len,
                                   len(sequences), self.n_hidden)[-1]

    # prediction on the value of the next time step (number of new cases the following day)                                
    y_pred = self.linear(last_time_step) ### CODE HERE ###

    return y_pred
  
  # Function needed to reset the hidden state  
  def reset_hidden_state(self):
    self.hidden = (
        torch.zeros(self.n_layers, self.seq_len, self.n_hidden),
        torch.zeros(self.n_layers, self.seq_len, self.n_hidden)
    )

We now have to define the function that handles the training of the LSTM-based model. As optimizer we'll use Adam with learning rate 1e-3, while as cost function we'll use MSELoss; we'll train the model for 100 epochs. We'll print the train and test loss values every 10 epochs.

In [5]:
def train_model(model, train_data, train_labels, test_data=None, 
                test_labels=None):
  
  loss_fn = torch.nn.MSELoss(reduction='sum')  # initializing the loss function
  
  optimiser = torch.optim.Adam(model.parameters(), lr=1e-3)  # initializing the optimizer
  
  num_epochs = 100
  
  train_loss_hist = np.zeros(num_epochs)  # training loss history
  test_loss_hist = np.zeros(num_epochs)  # testing loss history
  
  for t in range(num_epochs):

    model.reset_hidden_state() # resetting the hidden state
    
    y_pred = model(X_train)  # output on the sequence

    loss = loss_fn(y_pred.float(), y_train)  # loss calculation
    
    if test_data is not None:

      with torch.no_grad():

        y_test_pred = model(X_test)
        test_loss = loss_fn(y_test_pred.float(), y_test)
      
      test_loss_hist[t] = test_loss.item()

      if t % 10 == 0 or t==99:
        print(f"Epoch {t} train loss: {loss.item()}, "
          f"test loss: {test_loss.item()}")
    
    elif t % 10 == 0 or t==99:
      print(f'Epoch {t} train loss: {loss.item()}')
    
    train_loss_hist[t] = loss.item()

    optimiser.zero_grad()  # zeroing the gradients
    loss.backward()  # backward pass
    optimiser.step()  # optimization step

  return model.eval(), train_loss_hist, test_loss_hist

Let's now run our model.

In [6]:
model = TimeSeriesPredictor() ### CODE HERE ###

model, train_loss_hist, test_loss_hist = train_model(model, X_train, y_train, X_test, y_test) ### CODE HERE ###

Epoch 0 train loss: 54.07327651977539, test loss: 137.7874755859375
Epoch 10 train loss: 21.47332191467285, test loss: 72.49911499023438
Epoch 20 train loss: 17.60870361328125, test loss: 39.137428283691406
Epoch 30 train loss: 58.51490020751953, test loss: 1165.729736328125
Epoch 40 train loss: 58.866493225097656, test loss: 117.69242095947266
Epoch 50 train loss: 28.999780654907227, test loss: 92.35807800292969
Epoch 60 train loss: 21.2719669342041, test loss: 65.94776916503906
Epoch 70 train loss: 21.163148880004883, test loss: 64.65777587890625
Epoch 80 train loss: 21.05211067199707, test loss: 69.88024139404297
Epoch 90 train loss: 20.76102638244629, test loss: 66.59771728515625
Epoch 99 train loss: 20.585508346557617, test loss: 62.56349563598633
